In [1]:
# ================================================================
#  РЕГРЕССИЯ:  оценка площади патологической ткани (в пикселях)
#  Просто скопируйте‑вставьте и запустите
# ================================================================
import os, sys, json, cv2, torch, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from torch.utils.data import random_split, DataLoader, ConcatDataset
import torch.nn as nn
import segmentation_models_pytorch as smp

# ─────────────────────────────────────────────────────────────────
# 1.  Пути и импорт собственных модулей
# ─────────────────────────────────────────────────────────────────
seminar_path = Path("/home/alexskv/seminar_2")          # <‑‑ подстройте при необходимости
sys.path.insert(0, str(seminar_path / "core"))

from dataset import SimpleCocoDataset 
from trainers import SimpleRegressionTrainer               # trainer из предыдущего сообщения

# ─────────────────────────────────────────────────────────────────
# 2.  Параметры данных
# ─────────────────────────────────────────────────────────────────
resize      = (512, 512)
batch_size  = 32

pathology_ids = [i for i in range(6, 27) if i != 15]        # 6‑26, кроме 15
out_classes   = [                                           # только «Патология»
    {"id": 1, "name": "Патология",
     "summable_masks": pathology_ids,
     "subtractive_masks": []}
]

base_names = [
    "Правое лёгкое", "Левое лёгкое", "Контуры сердца", "Купола диафрагмы и нижележащая область",
    "Сложный случай", "нельзя составить заключение", "Иная патология", "Гидроторакс",
    "Легочно-венозная гипертензия 2 стадии и выше", "Пневмоторакс", "Доброкачественное новообразование",
    "Перелом ребра свежий", "Буллезное вздутие, тонкостенная киста", "Рак лёгкого (включая дорожку к корню при наличии)",
    "Кардиомегалия (отмечается всё сердце, как патология)", "Интерстициальная пневмония.",
    "Метастатическое поражение лёгких", "Полость с уровнем жидкости", "Грыжа пищевого отверстия диафрагмы",
    "Спавшийся сегмент лёгкого при ателектазе", "Инфильтративный туберкулёз",
    "Пневмония. В том числе сегментарная и полисегментарная", "Область распада, деструкции тканей лёгкого",
    "Участок пневмофиброза", "Кальцинаты. Каждый кальцинат выделяется отдельным контуром",
    "Консолидированный перелом ребра"
]
base_classes = [{"id": i + 1, "name": n} for i, n in enumerate(base_names)]

# ─────────────────────────────────────────────────────────────────
# 3.  Чтение всех task‑папок и DataLoader’ы
# ─────────────────────────────────────────────────────────────────
data_roots = {p.parent.parent for p in (seminar_path / "data")
              .rglob("annotations/instances_default.json")}
print(f"Найдено {len(data_roots)} датасетов:", *data_roots, sep="\n  ")

datasets = [
    SimpleCocoDataset(str(d), base_classes, out_classes, resize=resize)
    for d in sorted(data_roots)
]
full_ds = ConcatDataset(datasets)

val_pct   = 0.2
val_size  = int(len(full_ds) * val_pct)
train_ds, val_ds = random_split(full_ds, [len(full_ds) - val_size, val_size])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4)

# ─────────────────────────────────────────────────────────────────
# 4.  Модель: Unet‑encoder + суммирование площади
#     (предсказывает вероятность маски → суммируем → площадь)
# ─────────────────────────────────────────────────────────────────
class UnetAreaRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.unet = smp.Unet(
            encoder_name="efficientnet-b0",
            encoder_weights=None,
            in_channels=1,
            classes=1
        )

    def forward(self, x):
        logits = self.unet(x)                 # (N,1,H,W)
        probs  = torch.sigmoid(logits)
        area   = probs.sum(dim=(2, 3))        # (N,1) – площадь в пикселях
        return area

model = UnetAreaRegressor()

# ─────────────────────────────────────────────────────────────────
# 5.  Тренировка
# ─────────────────────────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainer = SimpleRegressionTrainer(
    model=model,
    classes=out_classes,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=10,
    exp_name="unet_area_regression"
)

trainer.train()


2025-04-14 12:07:48.038888: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-14 12:07:48.051777: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-14 12:07:48.055696: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 12:07:48.066934: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-14 12:07:48.710592: W tensorflow/compiler/tf2

Найдено 4 датасетов:
  /home/alexskv/seminar_2/data/task3
  /home/alexskv/seminar_2/data/task4
  /home/alexskv/seminar_2/data/task2
  /home/alexskv/seminar_2/data/task1
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


Train:  13%|█████████▉                                                                   | 4/31 [00:08<00:56,  2.08s/it]


KeyboardInterrupt: 